In [1]:
sc

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
14,application_1675528049135_0015,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<SparkContext master=yarn appName=livy-session-14>

In [2]:
import numpy as np
import pandas as pd
from pyspark.mllib.linalg.distributed import IndexedRowMatrix, IndexedRow


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
dim1 = 30
dim2 = 40
dim3 = 50

#A = np.random.randn(dim1, dim2)
#B = np.random.randn(dim2, dim3)

A = np.arange(dim1*dim2, dtype=np.float64).reshape(dim1, dim2)
B = np.arange(dim2*dim3, dtype=np.float64).reshape(dim2, dim3)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
def as_block_matrix(rdd, rowsPerBlock=5, colsPerBlock=5):
    return IndexedRowMatrix(
        rdd.zipWithIndex().map(lambda xi: IndexedRow(xi[1], xi[0]))
    ).toBlockMatrix(rowsPerBlock, colsPerBlock)

matrixA = as_block_matrix(sc.parallelize(A))
matrixB = as_block_matrix(sc.parallelize(B))
matrixC = matrixA.multiply(matrixB)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
C_np=np.matmul(A,B)
print(C_np[:2,:2])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[[1027000. 1027780.]
 [2587000. 2589380.]]

In [6]:
C_spark=matrixC.toLocalMatrix().toArray()
print(C_spark[:2,:2])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[[1027000. 1027780.]
 [2587000. 2589380.]]

In [7]:
print(np.allclose(C_np,C_spark))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

True

In [8]:
print(A.shape, len(matrixA.blocks.collect()))
print(B.shape, len(matrixB.blocks.collect()))
print(C_np.shape, len(matrixC.blocks.collect()))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(30, 40) 48
(40, 50) 80
(30, 50) 60

In [9]:
block_id=2
matrixC.blocks.collect()[block_id][1].toArray()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

array([[ 8827000.,  8835780.,  8844560.,  8853340.,  8862120.],
       [10387000., 10397380., 10407760., 10418140., 10428520.],
       [11947000., 11958980., 11970960., 11982940., 11994920.],
       [13507000., 13520580., 13534160., 13547740., 13561320.],
       [15067000., 15082180., 15097360., 15112540., 15127720.]])

In [10]:
C_np[5:10,0:5]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

array([[ 8827000.,  8835780.,  8844560.,  8853340.,  8862120.],
       [10387000., 10397380., 10407760., 10418140., 10428520.],
       [11947000., 11958980., 11970960., 11982940., 11994920.],
       [13507000., 13520580., 13534160., 13547740., 13561320.],
       [15067000., 15082180., 15097360., 15112540., 15127720.]])